## Notebook 2: Apprentissage automatique, Regression: Polution CO2

#### Importation des librairies

In [1]:
import pandas as pd
import numpy as np
#-------------------------------------------------
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.metrics import *
#-------------------------------------------------
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import TransformedTargetRegressor, make_column_transformer

#--------------------------------------------------------
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor

#--------------------------------------------------------
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.inspection import permutation_importance
#------------------------------------------------
from xgboost import XGBRegressor

#-----------------------------------------------------------
from sklearn.model_selection import GridSearchCV

## Checklist

Depuis 2001, **l’ADEME** acquiert tous les ans ces données auprès de **l’Union Technique de l’Automobile du motocycle et du Cycle UTAC** (en charge de l’homologation des véhicules avant leur mise en vente) en accord avec le ministère du développement durable.
Pour chaque véhicule les données d’origine (transmises par l’Utac) sont les suivantes :

* **Les consommations de carburant**

* **Les émissions de dioxyde de carbone (CO2)**

* **Les émissions des polluants de l’air** (réglementés dans le cadre de la norme Euro)

* **L’ensemble des caractéristiques techniques des véhicules** (gammes, marques, modèles, n° de CNIT, type d’énergie ...)



# L'inventaire des varaibles pertinentes:

Les données comprenent des variables pertinentes suivantes:

* **lib_mrq_utac**: La marque, il y'a 12.

* **lib_mod**: Le modèle commerciale, il y'a 20.

* **cod_cbr**: Le type de carburant, il y a 5.

* **hybride**: Information permettant d’identifier les véhicules hybrides (O/N)

* **puiss_max** : Puissance maximale

* **typ_boite_nb_rapp**: Type boite de vitesse et le nombre de rapport.

* **conso_urb**: Consommation urbaine de carburant (en l/100km),

* **conso_exurb**: consommation extra urbaine de carburant (en l/100km),

* **conso_mixte**: Consommation mixte de carburant (en l/100km),

* **co2**: Emission de CO2 (en g/km),

* **masse_ordma_min**: Masse en ordre de marche mini

* **masse_ordma_max**: Masse en ordre de marche max
 
* **Carrosserie**: Carrosserie

* **gamme**: Gamme du véhicule



# Objectif

Notre objectif majeur dans ce notebook 2 est de :

Prédire les emisisions de **CO2** des vehicules en fonction de certaines informations (Variables explicatives)

* En utilisant 4 à 5 modéles différents

* En comparant  les scores
    
* En choissisant le meilleur modèle


# Description des données

Lien vers les données: https://www.data.gouv.fr/fr/datasets/emissions-de-co2-et-de-polluants-des-vehicules-commercialises-en-france/


### Chargement, lecture, apercu et infos des données

In [2]:
data_model = pd.read_csv("data_model.csv")

In [3]:
unique_values = data_model['Carrosserie'].unique()
print(" : ",unique_values)

 :  ['BERLINE' 'BREAK' 'COUPE' 'CABRIOLET' 'TS TERRAINS/CHEMINS' 'COMBISPACE'
 'MINISPACE' 'MONOSPACE COMPACT' 'MONOSPACE' 'MINIBUS' 'COMBISPCACE']


### Selectionner les feautures les plus importans

In [4]:
df = data_model[['Carrosserie', 'masse_ordma_min', 'masse_ordma_max', 'co2']]

In [5]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()
objet_columns = df.select_dtypes(include='object').columns
for element in objet_columns:
    df[element]=le.fit_transform(df[element])

/tmp/ipykernel_174583/2377861051.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[element]=le.fit_transform(df[element])


### Traiter la colonne Carrosserie

In [6]:
df.head(10)

,Carrosserie,masse_ordma_min,masse_ordma_max,co2
0,0,1505.0,1505.0,182.0
1,0,1555.0,1555.0,186.0
2,0,1565.0,1565.0,134.0
3,0,1565.0,1565.0,134.0
4,0,1565.0,1565.0,139.0
5,0,1565.0,1565.0,139.0
6,0,1565.0,1565.0,136.0
7,1,1615.0,1615.0,137.0
8,1,1615.0,1615.0,137.0
9,1,1615.0,1615.0,142.0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55028 entries, 0 to 55027
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Carrosserie      55028 non-null  int64  
 1   masse_ordma_min  55028 non-null  float64
 2   masse_ordma_max  55028 non-null  float64
 3   co2              55028 non-null  float64
dtypes: float64(3), int64(1)
memory usage: 1.7 MB


### Prediction de CO2
    
Pour chacun de nos modèles:
   * **DummyRegressor**,
   * **LinearRegression**, 
   * **SGDRegressor**, 
   * **RandomForestRegressor**,
   * **GradientBoostingRegressor**,
   * **XGBRegressor**, 
    
Nous allons predire le niveau d'émission de **Co2** puis établir une comparaison entre les performances de chaque modèle.

    Appliquer un GridSearch pour optimiser les hyperparamètres de chaque modèle
    Paramètrer dans GridSearch les scores qui sont adaptés à la régression (R2, MAE et RMSE)
    Identifier le meilleur modèle

In [8]:
X = df.drop(['co2'] ,axis =1)
y = df['co2']

scaler = StandardScaler()

scaler.fit(X)

X = scaler.transform(X)

In [9]:
# Create the label encoder
le = LabelEncoder()

# Fit the label encoder to the target variable
le.fit(y)

# Transform the target variable
y= le.transform(y)

In [10]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, KFold

# Create the classifiers and their respective hyperparameter grids
classifiers = [DummyRegressor(),LinearRegression(),SGDRegressor(),RandomForestRegressor(),GradientBoostingRegressor(),XGBRegressor()]
param_grids = [
    {'strategy': ['mean', 'median'], 'constant': [0, 1]},
    {'fit_intercept': [True, False]},
    {'alpha': [0.0001, 0.001], 'penalty': ['l1', 'l2']},
    {'n_estimators': [50, 100, 200], 'max_depth': [None, 2, 4], 'min_samples_split': [3, 5, 7]},
    {'n_estimators': [50, 100], 'learning_rate': [0.1, 0.01], 'max_depth': [3, 5]},
    {'learning_rate': [0.1, 0.01], 'max_depth': [3, 5], 'n_estimators': [50, 80]}]

# Create the KFold object
#kf = KFold(n_splits=5)

# Create a list to store the results
results = []

# Loop through the classifiers and their hyperparameter grids
for i, (clf, param_grid) in enumerate(zip(classifiers, param_grids)):
    # Create the GridSearchCV object
    grid_search = GridSearchCV(clf, param_grid, scoring='r2', cv=5)
    # Fit the GridSearchCV object to the data
    grid_search.fit(X, y)
    # Store the results
    results.append((type(clf).__name__, grid_search.best_params_, grid_search.best_score_))
    # Print the best parameters and scores
    print("=============================================================")
    print(f"regreseur {i+1}: {type(clf).__name__}")
    print(f"best parameters: {grid_search.best_params_}")
    print(f"best score: {grid_search.best_score_}")
    print("=============================================================\n")

# Find the winner
winner = max(results, key=lambda x: x[2])

# Print the winner
print("=============================================================")
print(f"The winner is {winner[0]} with a score of {winner[2]} and parameters:")
print(winner[1])
print("=============================================================")

regreseur 1: DummyRegressor
best parameters: {'constant': 0, 'strategy': 'median'}
best score: -0.23304448159725616

regreseur 2: LinearRegression
best parameters: {'fit_intercept': True}
best score: 0.16237163660500403

regreseur 3: SGDRegressor
best parameters: {'alpha': 0.001, 'penalty': 'l1'}
best score: 0.17056888581811358

regreseur 4: RandomForestRegressor
best parameters: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 50}
best score: 0.2962822830499814

regreseur 5: GradientBoostingRegressor
best parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50}
best score: 0.3743341406617423

regreseur 6: XGBRegressor
best parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50}
best score: 0.36923339113021464

The winner is GradientBoostingRegressor with a score of 0.3743341406617423 and parameters:
{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 50}


In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

# Create the hyperparameter space
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 2, 4],
    'min_samples_split': [3, 5, 7],
}

# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(RandomForestRegressor(), param_grid, scoring='r2', cv=5)

# Fit the RandomizedSearchCV object to the data
random_search.fit(X, y)

# Print the best parameters and scores
print("=============================================================")
print(f"The winner is {random_search.best_estimator_} with a score of {random_search.best_score_}")
print("=============================================================")


The winner is RandomForestRegressor(min_samples_split=5) with a score of 0.28434952986120576


### Application Web

Développer une application Streamlit avec les options suivantes :

    1- L'utilisateur doit selectionner le type du carrosserie à partire d'un menu déroulant
    2- L'utilisateur doit saisir 'masse_ordma_min' et 'masse_ordma_max' dans deux champs de saisie différent
    3- Programmer un boutton pour lancer la prédiction de CO2

    masse_ordma_min = 825.0 - 2760.0
    masse_ordma_max = 825.0 - 3094.0
    Carrosserie = 0 - 10


    co2 = 13.0 - 572.0

In [12]:
#'Carrosserie', 'masse_ordma_min', 'masse_ordma_max'
df_data=[[10,1505.0,1505.0]]
model=random_search.best_estimator_
predictions = model.predict(df_data)

#donne toujour le meme resulta quand relancé
print(predictions)

[200.56118619]


In [13]:
from sklearn.linear_model import LinearRegression
df_test=[[]]
# Get the best regressor
regressor = winner[0]

# Get the best parameters
parameters = winner[1]

# Convert the parameters to a dictionary
parameters_dict = dict(parameters)

# Create a new regressor with the best parameters
if regressor == 'GradientBoostingRegressor':
    new_regressor = GradientBoostingRegressor(**parameters_dict)
else:
    # Handle other regressors accordingly
    pass

# Fit the new regressor to the data
new_regressor.fit(X, y)

# Get the predictions for new data
predictions = new_regressor.predict(df_data)

#donne des resulta peu diferant quand relancé
print(predictions)

[221.16292338]
